Initial Imports

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import warnings
import time
warnings.filterwarnings("ignore")
from datetime import date
import re
import requests
from scrapy.selector import Selector
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
myheaders = {'User-Agent': user_agent} 

STRIP GAME FUNCTION WITH PANDAS

In [2]:
# code to strip college basketball game stats using Pandas
def stripgame(gameid):

    printdf = True
    printdict = False

    # strip initial dataframes from espn url
    url = f'https://www.espn.com/mens-college-basketball/boxscore/_/gameId/{gameid}'
    # If the game has not started, blank dictionaries and lists will return
    try:
        game = pd.read_html(url)
    except Exception:
        print('GAME HAS NOT STARTED')
        return {}, {}, {}, [], []
        

    # total score
    totalscoredf = game[0]
    totalscoredf = totalscoredf.rename(columns={'Unnamed: 0': 'name'})
    hometeamname = totalscoredf.loc[0][0]
    awayteamname = totalscoredf.loc[1][0]
    totalscoredf = totalscoredf.rename(index={0: hometeamname})
    totalscoredf = totalscoredf.rename(index={1: awayteamname})
    totalscoredf = totalscoredf.drop(totalscoredf.columns[0], axis=1)
    if (printdf or printdict): print(f'{hometeamname} vs {awayteamname}')
    if printdf: print(totalscoredf)

    totalscoredict = totalscoredf.to_dict()
    if printdict: print(json.dumps(totalscoredict,indent=2))

    # Strip Hometeam player stats
    game[1] = game[1].reset_index()
    game[2] = game[2].reset_index()
    df = pd.merge(game[1], game[2], on='index')
    df = df.drop(columns='index')

    # reset header
    new_header = df.iloc[0]  # grab the first row for the header
    df = df[1:]  # take the data less the header row
    df.columns = new_header  # set the header row as the df header

    # drop column useless middle column
    df = df.drop([6]).reset_index().drop(columns='index')
    df = df.drop(index=df.index[-1], axis=0)

    # create better name list w/o position at the end
    homeplayernamelist = []
    for player in df['starters']:
        player = player.split(' ')[0] + ' ' + player.split(' ')[1]
        player = player.split(' ')[1] + ', ' + player.split(' ')[0]
        player = player.split('.')[0]
        #print(player)
        homeplayernamelist.append(player)
    playernamelistdf = pd.DataFrame(homeplayernamelist)
    playernamelistdf = playernamelistdf.reset_index()
    df = df.reset_index()
    df = pd.merge(playernamelistdf, df, on='index')
    df = df.drop(columns='index').drop(columns='starters')
    df = df.rename(columns={0: 'player'})

    hometeamplayerstats = df
    if printdf: print(hometeamplayerstats)

    # create dictionary of Hometeamplayerstats
    hometeamdict = hometeamplayerstats.transpose()
    new_header = hometeamdict.iloc[0]  # grab the first row for the header
    hometeamdict = hometeamdict[1:]  # take the data less the header row
    hometeamdict.columns = new_header  # set the header row as the df header
    hometeamdict = hometeamdict.to_dict()
    if printdict: print(json.dumps(hometeamdict,indent=2))

    # Strip Awayteam player stats
    game[3] = game[3].reset_index()
    game[4] = game[4].reset_index()
    df = pd.merge(game[3], game[4], on='index')
    df = df.drop(columns='index')

    # new header
    new_header = df.iloc[0]  # grab the first row for the header
    df = df[1:]  # take the data less the header row
    df.columns = new_header  # set the header row as the df header
    # drop useless column
    df = df.drop([6]).reset_index().drop(columns='index')
    df = df.drop(index=df.index[-1], axis=0)
    
    # create better name list w/o position at the end
    awayplayernamelist = []
    for player in df['starters']:
        player = player.split(' ')[0] + ' ' + player.split(' ')[1]
        player = player.split(' ')[1] + ', ' + player.split(' ')[0]
        player = player.split('.')[0]
        #print(player)
        awayplayernamelist.append(player)
    playernamelistdf = pd.DataFrame(awayplayernamelist)
    playernamelistdf = playernamelistdf.reset_index()
    df = df.reset_index()
    df = pd.merge(playernamelistdf, df, on='index')
    df = df.drop(columns='index').drop(columns='starters')
    df = df.rename(columns={0: 'player'})

    awayteamplayerstats = df
    if printdf: print(awayteamplayerstats)

    # create dictionary of Awayteamplayerstats
    awayteamdict = awayteamplayerstats.transpose()
    new_header = awayteamdict.iloc[0]  # grab the first row for the header
    awayteamdict = awayteamdict[1:]  # take the data less the header row
    awayteamdict.columns = new_header  # set the header row as the df header
    awayteamdict = awayteamdict.to_dict()
    if printdict: print(json.dumps(awayteamdict,indent=2))

    # extractgamestats
    comparisonurl = re.sub("boxscore", "matchup", url)
    game = pd.read_html(comparisonurl)
    df = game[1].rename(columns={'Unnamed: 1': f'{hometeamname}', 'Unnamed: 2': f'{awayteamname}'}).transpose()

    # reset header
    new_header = df.iloc[0]  # grab the first row for the header
    df = df[1:]  # take the data less the header row
    df.columns = new_header  # set the header row as the df header
    if printdf: print(df)
    if printdict: print(json.dumps(df.to_dict(),indent=2))

    if (printdict or printdf): print(homeplayernamelist)
    if (printdict or printdf): print(awayplayernamelist)

    return totalscoredict, hometeamdict, awayteamdict, homeplayernamelist, awayplayernamelist

#Example of code running and output
stripgame(401600298)

GAME HAS NOT STARTED


({}, {}, {}, [], [])

STRIP GAME FUNCTION WITHOUT PANDAS

In [4]:
#Code to strip game states w/o pandas

def stripgame(gameid):

    url = f'https://www.espn.com/mens-college-basketball/boxscore/_/gameId/{gameid}'

    #https://www.espn.com/mens-college-basketball/boxscore/_/gameId/401483406
    try:
        response = requests.get(url,headers=myheaders)
        content = response.content

        soup = BeautifulSoup(content, 'html.parser')
        tables = soup.find_all('table')
        rows = tables[0].find_all('tr')

    except IndexError:
        return {}, {}, {}, [], []

        #############################totalscoredict###########################################

    #############################totalscoredict###########################################
    totalscoredict={}
    rkey=""
    for row in rows:
        columns = row.find_all('td')
        for column in columns:

            if column.text.isdigit() == False:
                rkey=str(column.text)
                totalscoredict[rkey]=[]
            else:
                totalscoredict[rkey].append(column.text)

    key=dict.keys(totalscoredict)
    final_dic={}
    j=1
    for i in range (0,len(totalscoredict[rkey])):
        sub_dic={}
        for v in key:
            sub_dic[v]=totalscoredict[v][i]
        if j==len(totalscoredict[rkey]):
            final_dic[str("T")]=sub_dic
        else:
            final_dic[str(j)]=sub_dic

        j=j+1




    ######################################################################################



    #####################################hometeamdict#####################################
    rows = tables[1].find_all('tr')
    ply=[]
    for row in rows:
        columns = row.find_all('td')
        for column in columns:
            if " " in column.text:
                if column.text[1]==' ' or column.text[1]==".":
                    ply.append(column.text[3:-2]+", "+column.text[0])
                else:
                    ply.append(column.text[3:-2]+", "+column.text[0]+column.text[1])




    rows = tables[2].find_all('tr')
    list_v=[]

    for row in rows:
        columns = row.find_all('td')
        x=[]
        for column in columns:
            x.append(column.text)

        list_v.append(x)

    op=0
    inx=0
    for i in list_v:
        if op==0:
            pass
        else:
            if i[0].isalpha():
                inx=op
            else:
                pass
        op+=1
    del list_v[inx]



    hometeamdict={}
    for kk in range(0,len(ply)):
        sub_dic={}
        jj=0
        for ii in list_v[0]:
            sub_dic[ii]=list_v[kk+1][jj]
            jj+=1
        hometeamdict[ply[kk]]=sub_dic
    homeplayernamelist=ply

    ##############################################################################

    rows = tables[3].find_all('tr')
    ply=[]
    for row in rows:
        columns = row.find_all('td')
        for column in columns:
            if " " in column.text:
                if column.text[1]==' ' or column.text[1]==".":
                    ply.append(column.text[3:-2]+", "+column.text[0])
                else:
                    ply.append(column.text[3:-2]+", "+column.text[0]+column.text[1])






    rows = tables[4].find_all('tr')
    list_v=[]

    for row in rows:
        columns = row.find_all('td')
        x=[]
        for column in columns:
            x.append(column.text)

        list_v.append(x)
    op=0
    inx=0
    for i in list_v:
        if op==0:
            pass
        else:
            if i[0].isalpha():
                inx=op
            else:
                pass
        op+=1
    del list_v[inx]


    awayteamdict={}
    for kk in range(0,len(ply)):
        sub_dic={}
        jj=0
        for ii in list_v[0]:
            sub_dic[ii]=list_v[kk+1][jj]
            jj+=1
        awayteamdict[ply[kk]]=sub_dic
    awayplayernamelist=ply


    #print(final_dic, hometeamdict, awayteamdict, homeplayernamelist, awayplayernamelist)
    return final_dic, hometeamdict, awayteamdict, homeplayernamelist, awayplayernamelist
        #Example of code running and output: 
stripgame(401483313)

({'1': {'ILL': '37', 'TEX': '34'},
  '2': {'ILL': '31', 'TEX': '34'},
  '3': {'ILL': '17', 'TEX': '10'},
  'T': {'ILL': '85', 'TEX': '78'}},
 {'Hawkins, C': {'MIN': '42',
   'FG': '2-8',
   '3PT': '1-5',
   'FT': '4-6',
   'OREB': '2',
   'DREB': '5',
   'REB': '7',
   'AST': '4',
   'STL': '0',
   'BLK': '3',
   'TO': '1',
   'PF': '2',
   'PTS': '9'},
  'Mayer, M': {'MIN': '28',
   'FG': '8-10',
   '3PT': '5-5',
   'FT': '0-0',
   'OREB': '0',
   'DREB': '2',
   'REB': '2',
   'AST': '1',
   'STL': '1',
   'BLK': '3',
   'TO': '2',
   'PF': '4',
   'PTS': '21'},
  'Clark, S': {'MIN': '21',
   'FG': '2-5',
   '3PT': '1-2',
   'FT': '0-0',
   'OREB': '0',
   'DREB': '4',
   'REB': '4',
   'AST': '1',
   'STL': '0',
   'BLK': '0',
   'TO': '4',
   'PF': '3',
   'PTS': '5'},
  'Melendez, R': {'MIN': '27',
   'FG': '3-6',
   '3PT': '1-4',
   'FT': '2-2',
   'OREB': '0',
   'DREB': '4',
   'REB': '4',
   'AST': '0',
   'STL': '2',
   'BLK': '1',
   'TO': '2',
   'PF': '2',
   'PTS': '9'},
